# regression part 2 with Estimator API

In [15]:
import tensorflow as tf
import numpy as np
import pandas as pd
import ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d    

%matplotlib widget

In [16]:
data = pd.read_csv("Data/day_length_weight.csv")
print(data.head())
z_data = data['# day']
y_data = data['length']
x_data = data['weight']

other_data = data.drop('# day', axis=1)

    # day     length     weight
0   604.0  84.865512  13.030208
1   964.0  92.822120  14.719081
2  1640.0  98.521830  21.019549
3   642.0  85.633176  12.791008
4   467.0  75.294449  10.539456


In [17]:
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler

In [18]:
X_train, X_test, y_train, y_test = train_test_split(other_data, y_data, test_size=0.33, random_state=101)

In [19]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(data= scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(data= scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [20]:
length = tf.feature_column.numeric_column('length')
weight = tf.feature_column.numeric_column('weight')

In [21]:
feat_cols = [length, weight]
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs= 1000, shuffle=True)

In [22]:
model = tf.estimator.DNNRegressor(hidden_units=[2,4,10], feature_columns=feat_cols)

W0821 11:51:51.174730 4448298432 estimator.py:1811] Using temporary folder as model directory: /var/folders/rj/6zwt986d25zfxs3rksmpht2r0000gn/T/tmpk8xypigi


In [23]:
model.train(input_fn=input_func, steps=1000)

In [24]:
predict_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs= 1, shuffle=False)
pred_gen = model.predict(predict_fn)
predictions = list(pred_gen)

In [25]:
final_preds = []

for pred in predictions:
    final_preds.append(pred['predictions'])

# RMSE

In [26]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, final_preds)**0.5

3.454322124205659

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#ax.scatter(x_data, y_data, z_data, c='r', marker='o')

ax.scatter(X_test, y_test, final_preds, c='b', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: shape mismatch: objects cannot be broadcast to a single shape